In [48]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 1.4 MB/s eta 0:00:0000:0100:01
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.3 MB/s eta 0:00:0000:0100:01
  Using cached cloudpickle-2.2.0-py3-none-any.whl (25 kB)
  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 314.8 kB/s eta 0:00:0000:0100:04
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.0
    Uninstalling numpy-1.24.0:
      Successfully uninstalled numpy-1.24.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import sys
import os
import joblib
import shap

sys.path.append(os.path.dirname(os.getcwd()))

from dbn_nslkdd.dbn.tensorflow import SupervisedDBNClassification

Instructions for updating:
non-resource variables are not supported in the long term


In [157]:
# Load model
# if you have a pre-trained model the set to True, ealse set to False

LOAD_MODEL=True

In [158]:
DATA_DIR = '../data'
MODEL_DIR = '../models'
MODEL_NAME = os.path.join(MODEL_DIR, 'model_v1.pkl')

FILE_PATH_TRAIN_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_features.csv')
FILE_PATH_TRAIN_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_labels.csv')

FILE_PATH_TEST_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_features.csv')
FILE_PATH_TEST_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_labels.csv')

In [159]:
X_train_df = pd.read_csv(FILE_PATH_TRAIN_FEATURES)
Y_train_df = pd.read_csv(FILE_PATH_TRAIN_LABLES)

X_test_df = pd.read_csv(FILE_PATH_TEST_FEATURES).iloc[:, 1:]
Y_test_df = pd.read_csv(FILE_PATH_TEST_LABLES, header=None).iloc[1: , :]



In [160]:
X = np.array(X_train_df)
Y = np.array(Y_train_df).transpose()[0]

X_test = np.array(X_test_df)
# For prediction
Y_test = Y_test_df.transpose().to_numpy().tolist()[0]

In [161]:
n_epochs_rbm = 2
n_iter_backprop = 10

In [162]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=n_epochs_rbm,
                                         n_iter_backprop=n_iter_backprop,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)



In [163]:
if LOAD_MODEL:
    print(f'loading model from: {MODEL_NAME}')
    model = classifier.load(MODEL_NAME)
else:
    model = classifier.fit(X, Y)
    print(f'saving model to: {MODEL_NAME}')
    model.save(MODEL_NAME)


loading model from: ../models/model_v1.pkl


In [164]:
Y_pred = model.predict(X_test)


print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))



Done.
Accuracy: 0.804034
